<a href="https://colab.research.google.com/github/jonsol/small_projects/blob/master/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# To pd dataframe and change labels to numeric
train_dataset = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_dataset = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])
train_dataset['label'] = train_dataset['label'].map({'ham': 0, 'spam': 1})
test_dataset['label'] = test_dataset['label'].map({'ham': 0, 'spam': 1})


print(train_dataset)

In [ ]:
# Tokenize and Pad Text
vocab_size = 2000
max_length = 20
oov_token = "<OOV>"

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_dataset['message'])
sequences = tokenizer.texts_to_sequences(train_dataset['message'])
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding='post')

print(padded)

In [ ]:
# Model
model = Sequential([
    Embedding(vocab_size, 16, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')   #For binary classification?
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Train
history = model.fit(padded, train_dataset['label'], epochs=20, verbose=1)

In [ ]:
# Test
test_seq = tokenizer.texts_to_sequences(test_dataset['message'])
test_pad = tf.keras.preprocessing.sequence.pad_sequences(test_seq, maxlen=max_length, padding='post')

predictions = model.predict(test_pad)

for msg, pred in zip(test_dataset['message'], predictions):
    print(msg, "-->", "SPAM" if pred > 0.5 else "NOT SPAM")

In [ ]:
#pred_text = "how are you doing today?"

pred_text = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

pred_text = tokenizer.texts_to_sequences(pred_text)
pred_pad = tf.keras.preprocessing.sequence.pad_sequences(pred_text, maxlen=max_length, padding='post')

prediction_raw = model.predict(pred_pad)

prediction = []

for pred in prediction_raw:
  value = float(pred[0])
  label = "spam" if value > 0.5 else "ham"
  prediction.append((value, label))

print(prediction)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = tokenizer.texts_to_sequences(pred_text)
  pred_pad = tf.keras.preprocessing.sequence.pad_sequences(pred_text, maxlen=max_length, padding='post')

  prediction_raw = model.predict(pred_pad)

  prediction = []

  for pred in prediction_raw:
    value = float(pred[0])
    label = "spam" if value > 0.5 else "ham"
    prediction.append((value, label))

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
